# Data def etc

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/DNN')

In [2]:
import pandas as pd
import plotly.graph_objects as go
from rbflayer import RBFLayer, InitCentersRandom
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
######################
#Google authentication
######################

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [4]:
# Data reading

worksheet = gc.open('Data').sheet1
cols=['x','y']
table = pd.DataFrame(worksheet.get_all_values(),).T
table=pd.DataFrame(table.iloc[1:,:],dtype=float)
table.columns=cols
table
X=np.array(table['x']).reshape(-1,1)
y=np.array(table['y']).reshape(-1,1)
#print(X.shape,y.shape)
print(table)


      x       y
1  -0.9 -0.5770
2  -0.8 -0.0729
3  -0.7  0.3771
4  -0.6  0.6405
5  -0.5  0.6600
6  -0.4  0.4609
7  -0.3  0.1336
8  -0.2 -0.2013
9  -0.1 -0.4344
10  0.0 -0.5000
11  0.1 -0.3930
12  0.2 -0.1647
13  0.3  0.0988
14  0.4  0.3072
15  0.5  0.3960
16  0.6  0.3449
17  0.7  0.1816
18  0.8 -0.0312
19  0.9 -0.2189
20  1.0 -0.3201


In [5]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=X.reshape(-1),y=y.reshape(-1),mode='markers+lines'))
fig.show()

# Model

Defining RBF Layer

In [6]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [7]:
from tensorflow.keras.callbacks import Callback
import warnings
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [8]:
1497-50

1447

In [9]:
# Cross val split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=111)

In [28]:
%%time
## Defining network parametrs
hidden_units=16
loss_func='mse' # mean squared error loss function
opt=tf.keras.optimizers.SGD(0.2) # Stochastic gradient descent
                                 # method with learning rate 0.1
#opt=tf.keras.optimizers.Adagrad(0.05)
#opt=tf.keras.optimizers.Adam() #was the best
metric=['mae'] # additional metrics mean absolute error

######################

model = tf.keras.Sequential()
rbflayer = RBFLayer(hidden_units,
                        initializer=InitCentersRandom(X),
                        input_shape=(1,))

model.add(rbflayer)
model.add(tf.keras.layers.Dense(1,activation='linear')) # output layer with
                                                        # 1 neuron and linear
                                                        # activation func

model.compile(loss=loss_func,
              optimizer=opt,
              metrics=metric
              )

es=[tf.keras.callbacks.EarlyStopping(min_delta=0,monitor='val_loss',patience=50,mode='min')]
#es=[EarlyStoppingByLossVal(monitor='loss',value=0.0001,)] # stop training


history=model.fit(X_train, y_train,
          batch_size=10,
          epochs=5000,
          verbose=0,
          #callbacks=es,
          validation_data=(X_test,y_test)
          )

y_pred = model.predict(X)


fig=go.Figure()
fig.add_trace(go.Scatter(x=X.reshape(-1),y=y.reshape(-1),mode='lines+markers',name='Function'))
fig.add_trace(go.Scatter(x=X.reshape(-1),y=y_pred.reshape(-1),mode='lines+markers',name='Approximation',line=dict(dash='dash')))
fig.update_layout(width=1000,
                  margin=dict(l=20, r=20, t=20, b=20))
fig.show()

1/1 [==============================] - 0s 80ms/step


CPU times: user 2min 57s, sys: 5.6 s, total: 3min 2s
Wall time: 3min 23s


In [11]:
%cd 'drive/MyDrive/Colab Notebooks/DNN'

/content/drive/MyDrive/Colab Notebooks/DNN


In [ ]:
history.history

In [29]:
fig=go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'],
                         #mode='markers+lines',
                         name='Train loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'],
#                         mode='markers+lines',
                         name='Validation loss'))
fig.add_trace(go.Scatter(y=history.history['mae'],
#                         mode='markers+lines',
                         name='Training mae'))
fig.add_trace(go.Scatter(y=history.history['val_mae'],
#                         mode='markers+lines',
                         name='Validation mae'))

fig.update_layout(width=1000,
                  margin=dict(l=20, r=20, t=20, b=20))
fig.show()

In [18]:
history.history['loss'][-10:]

[7.260341226356104e-05,
 7.239192927954718e-05,
 7.435415318468586e-05,
 7.235116208903491e-05,
 7.43565324228257e-05,
 7.351835665758699e-05,
 7.321803423110396e-05,
 7.448163523804396e-05,
 7.283294689841568e-05,
 7.37562368158251e-05]

In [19]:
model.evaluate(X,y,batch_size=20)


1/1 [==============================] - 0s 44ms/step - loss: 6.1031e-05 - mae: 0.0059


[6.103108171373606e-05, 0.00588827021420002]

In [20]:
history.epoch[-1]

4999

In [31]:
model.predict([0.5])

1/1 [==============================] - 0s 124ms/step


array([[-1.0635903]], dtype=float32)

In [36]:
# Model perfomance with k-fold cross validation


#es=[EarlyStoppingByLossVal(monitor='val_loss',value=0.0001,)]
es=[tf.keras.callbacks.EarlyStopping(min_delta=0,monitor='val_loss',patience=50,mode='min')]


from sklearn.model_selection import KFold
folds_num=5
kfold = KFold(n_splits=folds_num, shuffle=True)
loss_per_fold=[]
val_loss_per_fold=[]
val_mae_per_fold=[]
histories=[]

for train, test in kfold.split(X,y):
  # model acrhitect
  model = tf.keras.Sequential()
  rbflayer = RBFLayer(hidden_units,
                        initializer=InitCentersRandom(X),
                        input_shape=(1,))
  opt=tf.keras.optimizers.Adagrad(0.05)
  model.add(rbflayer)
  model.add(tf.keras.layers.Dense(1,activation='linear'))
  # Compile
  model.compile(loss=loss_func,
              optimizer=opt,
              metrics=metric
              )
  # Fitting
  histories.append(model.fit(X[train], y[train],
          batch_size=5,
          epochs=5000,
          verbose=0,
          #callbacks=es,
          validation_data=(X[test],y[test])
          ))
  scores=model.evaluate(X[test],y[test],verbose=0,batch_size=5)
  loss_per_fold.append(scores[0])
  val_mae_per_fold.append(scores[1])



In [34]:
print(np.mean(loss_per_fold))
print(np.std(loss_per_fold))

0.0002799715188302798
0.00023651108693093845


In [ ]:
print(np.mean(val_mae_per_fold))
print(np.std(val_mae_per_fold))

In [37]:
fig=go.Figure()
#fig.add_trace(go.Scatter(y=histories[0].history['loss'],
                         #mode='markers+lines',
#                         name='Train loss fold1'))
fig.add_trace(go.Scatter(y=histories[0].history['val_loss'],
                         #mode='markers+lines',
                         name='Validation loss fold1'))

#fig.add_trace(go.Scatter(y=histories[1].history['loss'],
                         #mode='markers+lines',
#                         name='Train loss fold2'))
fig.add_trace(go.Scatter(y=histories[1].history['val_loss'],
                         #mode='markers+lines',
                         name='Validation loss fold2'))

#fig.add_trace(go.Scatter(y=histories[2].history['loss'],
                         #mode='markers+lines',
#                         name='Train loss fold3'))
fig.add_trace(go.Scatter(y=histories[2].history['val_loss'],
                         #mode='markers+lines',
                         name='Validation loss fold3'))

#fig.add_trace(go.Scatter(y=histories[3].history['loss'],
                         #mode='markers+lines',
#                         name='Train loss fold4'))
fig.add_trace(go.Scatter(y=histories[3].history['val_loss'],
                         #mode='markers+lines',
                         name='Validation loss fold4'))

#fig.add_trace(go.Scatter(y=histories[4].history['loss'],
                         #mode='markers+lines',
#                         name='Train loss fold5'))
fig.add_trace(go.Scatter(y=histories[4].history['val_loss'],
                         #mode='markers+lines',
                         name='Validation loss fold5'))
fig.update_layout(width=1000,
                  margin=dict(l=20, r=20, t=20, b=20))
fig.show()

In [ ]:
histories[2].epoch[-1]